<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
!pip install keras-tuner

     |████████████████████████████████| 61kB 4.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=c2309a6d339169fa909fa525f584483a0771b807b58790c750f80a92598e2ffa
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=b2c2c4b09335711b459a7bf3703659f2b862d356e92581494958eac81cbbd213
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


#建立一个标准模型

In [2]:
import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10
def build_model(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 256, step=32)
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
              filters, kernel_size=(3, 3), padding='same')(x)#filters卷积核
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D()(x)
        else:
            x = tf.keras.layers.AvgPool2D()(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
    x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
      optimizer=tf.keras.optimizers.Adam(
        hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
    return model


tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=4,
    hyperband_iterations=2)
#使用的是超频这个方法来实现找到最佳的超参数

data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']


def standardize_record(record):
    return tf.cast(record['image'], tf.float32) / 255., record['label']


train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).cache().batch(64)

tuner.search(train_ds,
             validation_data=test_ds,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

best_model = tuner.get_best_models(1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteZU2GD8/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteZU2GD8/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.
Epoch 1/2
782/782 [==============================] - 22s 29ms/step - loss: 1.2250 - accuracy: 0.5609 - val_loss: 1.4253 - val_accuracy: 0.5347
Epoch 2/2
782/782 [==============================] - 21s 27ms/step - loss: 0.7986 - accuracy: 0.7193 - val_loss: 0.8843 - val_accuracy: 0.6938


Epoch 1/2
782/782 [==============================] - 28s 36ms/step - loss: 1.7008 - accuracy: 0.3556 - val_loss: 1.5897 - val_accuracy: 0.4065
Epoch 2/2
782/782 [==============================] - 28s 36ms/step - loss: 1.2741 - accuracy: 0.5432 - val_loss: 1.8069 - val_accuracy: 0.4300


...

Epoch 1/4
782/782 [==============================] - 27s 35ms/step - loss: 1.7502 - accuracy: 0.3286 - val_loss: 1.7678 - val_accuracy: 0.3286
Epoch 2/4
782/782 [==============================] - 27s 34ms/step - loss: 1.3758 - accuracy: 0.4902 - val_loss: 1.3688 - val_accuracy: 0.5286
Epoch 3/4
782/782 [==============================] - 27s 34ms/step - loss: 1.1143 - accuracy: 0.6081 - val_loss: 1.0150 - val_accuracy: 0.6487
Epoch 4/4
782/782 [==============================] - 27s 34ms/step - loss: 0.9233 - accuracy: 0.6891 - val_loss: 1.0994 - val_accuracy: 0.6351


Epoch 1/4
782/782 [==============================] - 25s 33ms/step - loss: 1.2597 - accuracy: 0.5547 - val_loss: 1.2352 - val_accuracy: 0.5725
Epoch 2/4
782/782 [==============================] - 25s 32ms/step - loss: 0.7443 - accuracy: 0.7443 - val_loss: 1.0626 - val_accuracy: 0.6375
Epoch 3/4
782/782 [==============================] - 25s 32ms/step - loss: 0.5601 - accuracy: 0.8099 - val_loss: 0.7666 - val_accuracy: 0.7420
Epoch 4/4
782/782 [==============================] - 25s 32ms/step - loss: 0.4376 - accuracy: 0.8529 - val_loss: 0.6251 - val_accuracy: 0.7868


Epoch 1/4
782/782 [==============================] - 39s 50ms/step - loss: 1.4286 - accuracy: 0.4871 - val_loss: 1.2035 - val_accuracy: 0.5657
Epoch 2/4
782/782 [==============================] - 39s 50ms/step - loss: 0.9061 - accuracy: 0.6936 - val_loss: 1.1200 - val_accuracy: 0.6334
Epoch 3/4
782/782 [==============================] - 39s 50ms/step - loss: 0.7094 - accuracy: 0.7647 - val_loss: 0.9764 - val_accuracy: 0.7036
Epoch 4/4
782/782 [==============================] - 39s 50ms/step - loss: 0.5770 - accuracy: 0.8105 - val_loss: 0.9431 - val_accuracy: 0.6867


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
